In [1]:
import dask_awkward as dak
import awkward as ak
from distributed import LocalCluster, Client, progress
import time
import numpy as np
import matplotlib.pyplot as plt
import json
import mplhep as hep
import glob

plt.style.use(hep.style.CMS)

client =  Client(n_workers=15,  threads_per_worker=1, processes=True, memory_limit='8 GiB') 


In [2]:
def getHist(events, field2plot, binning):
    weight = ak.fill_none(events.wgt_nominal_total, value=0)
    value = ak.fill_none(events[field2plot], value=-999)
    # use np.isnan to filter away remaining nan values
    nan_filter = ~(np.isnan(weight) | np.isnan(value)) # some nans are not None, apparently
    weight = weight[nan_filter]
    value = value[nan_filter]
    
    print(f"getHist weight sum: {np.sum(weight)}")
    print(f"getHist value sum: {np.sum(value)}")
    print(f"getHist weight: {weight}")
    print(f"getHist value: {value}")
    # print(f"getHist is none weight: {ak.sum(ak.is_none(weight))}")
    # print(f"getHist is none value: {ak.sum(ak.is_none(value))}")
    # weight = weight/ np.sum(weight) # normalize to one
    # print(f"np.sum(weight): {np.sum(weight)}")
    hist, edges = np.histogram(value, bins=binning, weights=weight)
    print(f"getHist hist b4 normalization: {hist}")
    hist = hist / np.sum(hist)
    print(f"np.sum(hist): {np.sum(hist)}")
    return hist, edges

def applyGGH_cut(events):
    btag_cut =ak.fill_none((events.nBtagLoose >= 2), value=False) | ak.fill_none((events.nBtagMedium >= 1), value=False)
    # vbf_cut = ak.fill_none(events.vbf_cut, value=False
    vbf_cut = (events.jj_mass > 400) & (events.jj_dEta > 2.5) 
    vbf_cut = ak.fill_none(vbf_cut, value=False)
    region = events.h_sidebands | events.h_peak
    # region =  events.h_peak # whether just h_peak or signal region, the plots don't change
    ggH_filter = (
        ~vbf_cut & 
        region &
        ~btag_cut # btag cut is for VH and ttH categories
    )
    return events[ggH_filter]

def getDeltaPhi(phi1,phi2):
    phi1 = ak.ones_like(phi1, dtype="double")*phi1
    phi2 = ak.ones_like(phi2, dtype="double")*phi2
    dphi = abs(np.mod(phi1 - phi2 + np.pi, 2 * np.pi) - np.pi)
    return dphi

def computeBkgFromParquet(load_path, bkgSample_l, fields2compute):
    zip_l = []
    # fields2compute =  fields2compute +["wgt_nominal_zpt_wgt"]
    for sample in bkgSample_l:
        events = dak.from_parquet(load_path+f"/{sample}/*/*.parquet")
        # print(events.fields)
        # print(events.wgt_nominal_zpt_wgt)
        events["jj_dRapidity"] = np.abs(events.jet1_rapidity - events.jet2_rapidity)
        events["mmj1_dRapidity"] = np.abs(events.jet1_rapidity - events.dimuon_rapidity)
        events["mmj2_dRapidity"] = np.abs(events.jet2_rapidity - events.dimuon_rapidity)
        events["jj_dPhiV2"] = ak.fill_none(getDeltaPhi(events.jet1_phi, events.jet2_phi), value=-1)
        bool_filter = ak.fill_none((events.mmj1_dEta < events.mmj2_dEta), value=True)
        events["mmj_min_dEtaV2"] = ak.where(bool_filter, events.mmj1_dEta, events.mmj2_dEta)
        bool_filter = ak.fill_none((events.mmj1_dPhi < events.mmj2_dPhi), value=True)
        events["mmj_min_dPhiV2"] = ak.where(bool_filter, events.mmj1_dPhi, events.mmj2_dPhi)
        zip = ak.zip({field: events[field] for field in fields2compute}).compute()
        zip_l.append(zip)
    
    final_zip = ak.concatenate(zip_l)
    # zpt removal test start ------------------------
    # final_zip["wgt_nominal_total"] = final_zip.wgt_nominal_total / final_zip.wgt_nominal_zpt_wgt
    # zpt removal test end ------------------------
    return final_zip



In [3]:
training_samples = {
        "background": [ # for some reason, having more than dy causes things to break
            "dy_M-100To200", 
            "ttjets_dl",
            "ttjets_sl",
            "st_tw_top",
            "st_tw_antitop",
            "ww_2l2nu",
            "wz_1l1nu2q",
            "wz_2l2q",
            "wz_3lnu",
            "zz",
            "ewk_lljj_mll50_mjj120",
        ],
        "signal": [
            "ggh_powheg", 
            "vbf_powheg",
    ],
}

In [4]:
# year = 2018
# load_path = f"/depot/cms/users/yun79/results/stage1/DNN_test2/{year}/f1_0"
# load_path = f"/depot/cms/users/yun79/results/stage1/BDT_inputValidation/{year}/f1_0"
# load_path = f"/depot/cms/users/yun79/results/stage1/BDT_inputValidation_V2/{year}/f1_0"
# load_path = f"/depot/cms/users/yun79/results/stage1/BDT_inputValidation_V2/*/f1_0"
load_path = f"/depot/cms/users/yun79/results/stage1/BDT_inputValidation_JetIdUpdate/*/f1_0"
# load_path = f"/depot/cms/users/yun79/results/stage1/BDT_inputValidation_JetIdUpdate/2018/f1_0"
# bkg_l = [load_path+f"/{name}/*/*.parquet" for name in training_samples["background"]]
# bkg_l = []
# for bkg in training_samples["background"]:
#     bkg_l += glob.glob(load_path+f"/{bkg}/*/*.parquet")
# simple from parquet doesn't work on bkg, so special care is needed

# bkg_l
# sig_l = [load_path+f"/{name}/*/*.parquet" for name in training_samples["signal"]]
# sig_l

In [5]:
start_time = time.time()
cols_of_interest = [
    # 'dimuon_rapidity',
    # 'dimuon_cos_theta_cs',
    # 'dimuon_phi_cs',
    # 'dimuon_pt',
    'jet1_eta',
    'jet1_pt',
    'jet2_pt',
    # 'jet2_eta', # for test purpose
    'jj_dEta',
    'jj_dPhi',
    'jj_dPhiV2',
    'jj_mass',
    # 'mmj1_dEta',
    # 'mmj1_dPhi',
    # 'mmj_min_dEta',
    # 'mmj_min_dPhi',
    # 'mmj_min_dEtaV2',
    # 'mmj_min_dPhiV2',
    # 'mu1_eta',
    # 'mu1_pt_over_mass',
    # 'mu2_eta',
    # 'mu2_pt_over_mass',
    # 'zeppenfeld',
    # 'njets',
    # "jj_dRapidity", # for test purpose
    # 'mmj1_dRapidity', # for test purpose
    # 'mmj2_dRapidity', # for test purpose
    # 'mmj2_dEta',# for test purpose
    # 'mmj2_dPhi',# for test purpose
]
additional_fields = [
    "wgt_nominal_total",
    "h_sidebands",
    "h_peak",
    "vbf_cut",
    "nBtagLoose",
    "nBtagMedium",
    # "mu1_pt",
    # "mu2_pt",
    "dimuon_mass",
    "jet1_rapidity",
    "jet2_rapidity",
    "jet1_phi",
    "jet2_phi",
]
fields2compute = cols_of_interest +  additional_fields


# events_bkg = dak.from_parquet(bkg_l) 
# events_bkg = ak.zip({field : events_bkg[field] for field in fields2compute}).compute()

# normal from_parquet doesn't work, so using convoluted concatenating method

events_bkg = computeBkgFromParquet(
    load_path, 
    training_samples["background"], 
    fields2compute,
)


events_bkg = applyGGH_cut(events_bkg)

#calculate sig_l
sig_l = [load_path+f"/{name}/*/*.parquet" for name in training_samples["signal"]]
events_sig = dak.from_parquet(sig_l)
events_sig["jj_dRapidity"] = np.abs(events_sig.jet1_rapidity - events_sig.jet2_rapidity)
events_sig["mmj1_dRapidity"] = np.abs(events_sig.jet1_rapidity - events_sig.dimuon_rapidity)
events_sig["mmj2_dRapidity"] = np.abs(events_sig.jet2_rapidity - events_sig.dimuon_rapidity)
events_sig["jj_dPhiV2"] = ak.fill_none(getDeltaPhi(events_sig.jet1_phi, events_sig.jet2_phi), value=-1)
bool_filter = ak.fill_none((events_sig.mmj1_dEta < events_sig.mmj2_dEta), value=True)
events_sig["mmj_min_dEtaV2"] = ak.where(bool_filter, events_sig.mmj1_dEta, events_sig.mmj2_dEta)
bool_filter = ak.fill_none((events_sig.mmj1_dPhi < events_sig.mmj2_dPhi), value=True)
events_sig["mmj_min_dPhiV2"] = ak.where(bool_filter, events_sig.mmj1_dPhi, events_sig.mmj2_dPhi)



# events_sig["wgt_nominal_total"].compute()
events_sig = ak.zip({field : events_sig[field] for field in fields2compute}).compute()
events_sig = applyGGH_cut(events_sig)

print(time.time()-start_time)


370.8627498149872


In [6]:
# isnan = np.isnan(ak.to_numpy(events_bkg.wgt_nominal_total))
# np.sum(isnan)
# # hist_bkg

In [8]:
# nbins = 60
# bin_map = {
#      "dimuon_pt": [0,200, nbins], 
#     "dimuon_rapidity" : [-2.5,2.5, nbins], 
#     "dimuon_eta" : [-8,8, nbins],
# }
with open("./plot_settings_gghCat_BDT_input.json", "r") as file:
    bin_map = json.load(file)
# bin_map

In [9]:
year = "Run2"
# for field in cols_of_interest:
for field in (["jj_dPhiV2"]):
    binning = np.linspace(*bin_map[field]["binning_linspace"])
    xmin = bin_map[field]["binning_linspace"][0]
    xmax = bin_map[field]["binning_linspace"][1]
    hist_sig, edges = getHist(events_sig, field, binning)
    # raise ValueError
    hist_bkg, edges = getHist(events_bkg, field, binning)
    fig, ax_main = plt.subplots()
    # plt.stairs(hist_sig,edges=edges,label="signal", color="blue")
    # plt.stairs(hist_bkg,edges=edges,label="background", color="red")
    hep.histplot(
        hist_sig, 
        bins=binning, 
        stack=False, 
        histtype='step', 
        color='blue', 
        label='signal', 
        ax=ax_main,
    )
    # print(f"hist_bkg: {hist_bkg}")
    hep.histplot(
        hist_bkg, 
        bins=binning, 
        stack=False, 
        histtype='step', 
        color='red', 
        label='background', 
        ax=ax_main,
    )
    ax_main.set_xlabel(bin_map[field]["xlabel"])
    ax_main.set_ylabel("A.U.")
    if bin_map[field]["logscale"]:
        plt.yscale('log')  # Set y-axis to log scale
        plt.ylim(1e-3, 1)
    plt.xlim(xmin, xmax)
    plt.legend()
    # plt.show()
    CenterOfMass = 13
    # lumi = 59.97 # 2018 lumi value
    lumi = 137.9 # Run2 value
    hep.cms.label(data=True, loc=0, label="Private Work", com=CenterOfMass, ax=ax_main, lumi=lumi)
    plt.savefig(f"plots/BDT_input{year}_{field}")
    plt.clf()

getHist weight sum: 881.5309439082096
getHist value sum: -971341.2513500757
getHist weight: [0.000379, 0.000224, 0.000261, 0.000312, ..., 2.23e-05, 2.45e-05, 3.51e-05]
getHist value: [-1, -1, -1, -1, -1, -1, -1, -1, ..., -1, 2.95, -1, -1, 0.453, -1, -1, 0.521]
getHist hist b4 normalization: [2.52, 2.54, 2.68, 2.65, 2.78, 2.8, ..., 3.19, 3.17, 3.3, 3.34, 3.51, 3.54]
np.sum(hist): 1.0
getHist weight sum: 1737178.0372940407
getHist value sum: -20602466.672882587
getHist weight: [0.0293, 0.0325, 0.0273, 0.0258, -0.0255, ..., 0.0103, 0.0119, 0.0113, 0.0143]
getHist value: [-1, -1, -1, -1, -1, -1, -1, -1, -1, ..., 2.33, -1, -1, -1, 1.49, 1.8, -1, 2.28]
getHist hist b4 normalization: [2.52e+03, 2.6e+03, 2.57e+03, 2.66e+03, ..., 7.31e+03, 7.59e+03, 7.93e+03]
np.sum(hist): 1.0


<Figure size 1000x1000 with 0 Axes>

In [ ]:
ak.sum(ak.is_none(events_bkg.jj_dPhiV2))

In [ ]:
A = ak.Array([2.0], dtype="double")
ak.types.NumpyType.copy(A)